In [4]:
from lxml import etree, html as lhtml
import requests
from bs4 import BeautifulSoup
import sys
import json
from urllib.parse import urljoin
from urllib import parse
from tqdm.notebook import tqdm
from multiprocessing.dummy import  Queue
from multiprocessing import Pool
import gzip
import codecs
import json
import os

In [5]:
def get_page(url, n_attempts = 3, t_sleep = 0.1):
    for i in range(n_attempts):
        page = requests.get(url)
        if not page.ok:
            print(page.status_code, file=sys.stderr)
        else:
            return page
    time.sleep(t_sleep)
    print("Error in getting {} page".format(url), file=sys.stderr)
    return None

In [6]:
def get_all_pages(сount):
    url = get_page("https://gg.deals/games/?sort=metascore&type=1&page={}".format(count))
    if url is None:
        print('bad')
        return None
    tree = lhtml.fromstring(url.text)
    for i in range(1, 13):
        for link in tree.xpath('//*[@id="games-list"]/div[1]/div[1]/div[{}]/a[1]/@href'.format(i)):
            links.append('https://gg.deals' + link)

In [7]:
links = []
for count in range(1, 26):
    get_all_pages(count)

In [8]:
def get_pages(сount):
    url = get_page("https://gg.deals/games/?sort=metascore&type=1&page={}".format(count))
    if url is None:
        print('bad')
        return None
    temp = []
    tree = lhtml.fromstring(url.text)
    for i in range(1, 13):
        for link in tree.xpath('//*[@id="games-list"]/div[1]/div[1]/div[{}]/a[1]/@href'.format(i)):
            temp.append('https://gg.deals' + link)
    return temp

In [9]:
links = [] #страницы
for count in range(1, 26):
    links.append(get_pages(count))
print(len(links))

25


In [10]:
def get_name(tree):
    link = tree.xpath('//*[@id="page"]/div[2]/div/ul/li[4]/a/span//text()')
    return link[0]

In [11]:
def get_image(soup):
    imgs = soup.find_all('img', {'class': 'image-blur'})
    for img in imgs:
        image = img.get('src')
    return image

In [12]:
def get_release_date(block):
    c = block.find('div', {'class': 'game-info-details'})\
             .find('p', {'class': 'game-info-details-content'}).text
#     print(c)
    return c

In [13]:
def get_developer(block):
    c = block.find('div', {'class': 'game-info-details-section game-info-details-section-developer'})\
         .find('p', {'class': 'game-info-details-content'}).text
#     print(c)
    return c

In [14]:
def get_scores(block):
    temp = {}
    temp_score= block.find('div', {'class': 'game-info-details'})\
         .find('div', {'class': 'game-info-details-content'})\
         .find_all('span', {'class': 'overlay'})
    if temp_score is not None:
        try:
            temp['metacritic_score'] = (int(temp_score[0].text))
        except IndexError:
            temp['metacritic_score'] = None
        try:
            temp['user_score'] = (float(temp_score[1].text))
        except IndexError:
            temp['user_score'] = None
    return temp

In [15]:
def get_reviews(block):
    reviews = {}
    try:
        reviews = block.find('div', {'class': 'game-info-details'})\
             .find('div', {'class': 'game-info-details-content'})
        rew = reviews.find('span', {'class': 'reviews-label'}).text
        beg_ = rew.find(' (')
        review_label = rew[:beg_]
        end_ = rew.find(')')
        review_count = rew[beg_ + 2:end_].replace(',', "")
        review_count = int(review_count)
        string = reviews.find('span', {'class': 'reviews-label'})['title']
        ind = string.find('%')
        review_positive_pctg = int(string[:ind])
        reviews['review_label'] = review_label
        reviews['review_positive_pctg'] = review_positive_pctg
        reviews['review_count'] = review_count
    except AttributeError:
        reviews = None
    return reviews

In [16]:
def get_genres(block):
    genres = []
    temp = block.find('div', {'class': 'game-info-genres tags-container'})\
            .find_all('a' , {'class': 'badge'})
    for x in temp:
        genres.append(x.text)
    return genres

In [17]:
def get_tags(block):
    tags = []
    try:
        temp = block.find('div', {'id': 'game-info-tags'})\
                .find_all('a' , {'class': 'badge'})
    except AttributeError:
        return None 
    if temp is not None:
        for x in temp:
            tags.append(x.text)
    return tags

In [18]:
def get_features(block):
    features = []
    try:
        temp = block.find('div', {'id': 'game-info-features'})\
                .find_all('a' , {'class': 'badge'})
    except AttributeError:
        return None 
    if temp is not None:
        for x in temp:
            features.append(x.text)
    return features

In [19]:
def get_pc_systems(soup):
    pc_systems = []
    try:
        block = soup.find('div', {'class': 'game-section bg-light game-about'})\
                .find('ul', {'class': 'nav'})\
                .find_all('a', {'role': 'button'})
    except AttributeError:
        return None 
    if block is not None:
        for x in block:
            pc_systems.append(x.text)
    return pc_systems

In [20]:
def get_counts(tree):
    temp = []
    for i in range(1, 4):
        for x in tree.xpath("/html/body/div[1]/div[4]/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/div/div[{}]/span[1]/span[3]/span[2]".format(i)):
            temp.append(x.text)
    wishlist_count = temp[0]
    alert_count = temp[1]
    owners_count = temp[2]
    return temp

In [21]:
def get_dlcs(block):
    dlcs = []
    try:
        temp = block.find('section', {'id': 'game-dlcs'})\
                .find('div', {'class': 'games-box games-box-related-list load-more-content'})\
                .find_all('a', {'class': 'full-link'})
    except AttributeError:
        return None
    for x in temp:
        if 'href' in x.attrs:
            dlcs.append('https://gg.deals' + x['href'])
    return dlcs

In [22]:
def get_full_dlcs(block2):
    dlcs = []
    temp = block2.find_all('a', {'class': 'full-link'})
    for x in temp:
        if 'href' in x.attrs:
            dlcs.append('https://gg.deals' + x['href'])
    return dlcs

In [23]:
def get_packs(block):
    try:
        temp = block.find('section', {'class': 'tab-menu-section offer-section', 'id': 'game-packs'})\
                .find('div', {'class': 'games-box games-box-related-list load-more-content'})\
                .find_all('a', {'class': 'full-link'})
    except AttributeError:
        return None
    packs = []
    for x in temp:
        if 'href' in x.attrs:
            packs.append('https://gg.deals' + x['href'])
    return packs

In [24]:
def get_full_packs(block2):
    packs = []
    temp = block2.find_all('a', {'class': 'full-link'})
    for x in temp:
        if 'href' in x.attrs:
            packs.append('https://gg.deals' + x['href'])
    return packs

In [25]:
def get_market_url(tree):
    try:
        link = tree.xpath('//*[@id="game-card"]/div[1]/div/div[1]/div/div[1]/div/a/@href')
    except AttributeError:
        return None
    if link:
        page = requests.get(link[0])
        return page.url
    else:
        return None

In [26]:
def get_price_history(link, soup):
    try:
        chart_link = soup.find('div', {'id': 'historical-chart-container'})['data-with-keyshops-url']
        full_chart_url = urljoin(link, chart_link)
        headers = {'X-Requested-With':'XMLHttpRequest'}
        web1 = requests.get(full_chart_url, headers=headers)
        answer = json.loads(web1.text)
        data = answer['chartData']['deals']
        price_history = []
        for i in range(0, len(data)):
            price_dict = {}
            price_dict['ts'] = data[i]['x']
            price_dict['price'] = data[i]['y']
            price_dict['shop'] = data[i]['shop']
            price_history.append(price_dict)
        return price_history
    except TypeError:
        return None

In [27]:
def process_page(link):
    temp = {}
    url = None
    try:
        url = get_page(link)
        tree = lhtml.fromstring(url.text)
        soup = BeautifulSoup(url.text, 'lxml')
    except:
        print("Error in getting {} page".format(link), file=sys.stderr)
        return None
    info = {}
    info['url'] = link
    get_value = get_name(tree)
    if url is not None:
        info['status'] = True
    if get_value is not None:
        info['name'] = get_value
    get_value = get_image(soup)
    if get_value is not None:
        info['image'] = get_value
    #right block
    block = soup.find('div', {'class': 'game-section game-offers'})\
        .find('div', {'class': 'col-right'})
    get_value = get_release_date(block)
    if get_value is not None:
        info['release_date'] = get_value
    get_value = get_developer(block)
    if get_value is not None:
        info['developer'] = get_value
    get_value = get_scores(block)
    if get_value is not None:
        try:
            info['metacritic_score'] = get_value['metacritic_score']
        except KeyError:
            pass
        try:
            info['user_score'] = get_value['user_score']
        except KeyError:
            pass
    get_value = get_reviews(block)
    if get_value is not None:
        info['review_label'] = get_value['review_label']
        info['review_positive_pctg'] = get_value['review_positive_pctg']
        info['review_count'] = get_value['review_count']
    get_value = get_genres(block)
    if get_value is not None:
        info['genres'] = get_value
    get_value = get_tags(block)
    if get_value is not None:
        info['tags'] = get_value
    get_value = get_features(block)
    if get_value is not None:
        info['features'] = get_value
    get_value = get_pc_systems(soup)
    if get_value is not None:
        info['pc_systems'] = get_value
    get_value = get_counts(tree)
    if get_value is not None:
        info['wishlist_count'] = get_value[0]
        info['alert_count'] = get_value[1]
        info['owners_count'] = get_value[2]
        
        
#     block have changed!!!    
    block = soup.find('div', {'class': 'game-section game-offers'})\
            .find('div', {'class': 'col-left'})
    get_value = get_dlcs(block)
    if get_value is not None:    
        info['dlcs'] = get_value
    try:
        block2 = soup.find('div', {'class': 'game-related-container list-more-wrapper list-more-ajax-wrapper load-more-container'})
        get= block2.find('a', {'class': 'widget-link-more news-list-show-more-link load-more-button'})
        response = get['data-url']
        main_content = urljoin(link, response)
        page2 = requests.get(main_content,headers = {'X-Requested-With':'XMLHttpRequest'})
        soup2 = BeautifulSoup(page2.text, 'lxml')
        get_value = get_full_dlcs(soup2)
        if get_value is not None:    
            info['dlcs'] += get_value
    except:
        pass
        
    get_value = get_packs(block)
    if get_value is not None:    
        info['packs'] = get_value
    try:
        block2 = soup.find('section', {'class': 'tab-menu-section offer-section', 'id': 'game-packs'})\
                .find('div', {'class': 'game-related-container list-more-wrapper list-more-ajax-wrapper load-more-container'})
        get= block2.find('a', {'class': 'widget-link-more news-list-show-more-link load-more-button'})
        response = get['data-url']
        main_content = urljoin(link, response)
        page2 = requests.get(main_content,headers = {'X-Requested-With':'XMLHttpRequest'})
        soup2 = BeautifulSoup(page2.text, 'lxml')
        get_value = get_full_packs(soup2)
        if get_value is not None:    
            info['packs'] += get_value
    except:
        pass
    
    
    get_value = get_market_url(tree)
    if get_value is not None:    
        info['market_url'] = get_value        
    get_value = get_price_history(link, soup)
    if get_value is not None:    
        info['price_history'] = get_value        
    
    return info
    

In [28]:
Queue = []
for a in links:
    for i in a:
        Queue.append(i)

In [29]:
print(len(Queue))

300


In [32]:
with tqdm(total=len(Queue)) as pbar:
    lock = pbar.get_lock()
    with Pool(processes=15) as pool:
        for i, info in enumerate(pool.imap(process_page, Queue)):
            with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
                f_json = codecs.getwriter('utf8')(f_json)
#                 print(Queue[i])
                if info is not None:
                    info_str = json.dumps(info, ensure_ascii=False)
                    print(info_str, file=f_json)
                else:
                    print("error", file=sys.stderr)
                with lock:
                    pbar.update(1)
pool.join()